In [1]:
from bs4 import BeautifulSoup
import urllib2
import csv

BASE_URL = "http://androidweekly.net/issues/issue-{}"

In [2]:
def contentForIssue(num):
    url = BASE_URL.format(num)
    print url
    request = urllib2.Request(url)
    request.add_header('Accept-Encoding', 'utf-8')
    response = urllib2.urlopen(url)
    soup = BeautifulSoup(response.read().decode('utf-8'), 'lxml')
    content = soup.find('div', { 'class':'issue' })
    return content

In [72]:
def dataForIssue(content, issue_num):
    sections = content.find_all('td')
    last_section_header = ""
    data = []
    section_count = 0
    for section in sections:
        if section.h2 or section.h3:
            section_count += 1
            if section.h2:
                last_section_header = section.h2.text
            else:
                last_section_header = section.h3.text
        if section.a and section.a.string:
            item = {}
            item['issue_num'] = issue_num
            item['section'] = last_section_header.encode("utf-8")
            item['title'] = section.a.text.encode("utf-8").replace('\n','')
            item['link'] = section.a.get('href').encode("utf-8")
            item['site'] = section.span.text.replace('(','').replace(')', '').replace('\n','').encode("utf-8")
            item['description'] = section.p.text.replace('\n','').encode("utf-8")
            data.append(item)
    print 'Found %d items in %s sections' % (len(data), section_count)
    return data

In [137]:
def dataForIssueBefore103(content, issue_num):
    sections = content.find_all(['h2', 'div', 'p'])
    last_section_header = ""
    data = []
    section_count = 0
    for section in sections:
        if section.name == 'h2':
            section_count += 1
            last_section_header = section.text
        elif section.name == 'div' and len(section.find_all('a')) > 1:
            links = section.find_all('a');
            item = {}
            item['issue_num'] = issue_num
            item['section'] = last_section_header.encode("utf-8")
            item['title'] = links[1].string.encode("utf-8").replace('\n','')
            item['link'] = links[1].get('href').encode("utf-8")
            item['site'] = section.span.text.replace('(','').replace(')', '').replace('\n','').encode("utf-8")
            item['description'] = section.p.text.replace('\n','').encode("utf-8")
            data.append(item)
        elif section.name == 'p' and section.a and section.a.text:
            item = {}
            item['issue_num'] = issue_num
            item['section'] = last_section_header.encode("utf-8")
            item['title'] = section.a.string.encode("utf-8").replace('\n','')
            item['link'] = section.a.get('href').encode("utf-8")
            if section.span:
                item['site'] = section.span.text.replace('(','').replace(')', '').replace('\n','').encode("utf-8")
            else:
                item['site'] = ""
            if section.br and section.br.next_sibling:
                item['description'] = section.br.next_sibling.replace('\n','').encode("utf-8")
            else:
                item['description'] = ""
            data.append(item)
    print 'Found %d items in %s sections' % (len(data), section_count)
    return data

In [131]:
def dataForIssueBefore60(content, issue_num):
    return dataForIssueBefore103(content.div, issue_num)

In [4]:
def writeData(data, fname):
    with open(fname, 'wb') as f:
        w = csv.DictWriter(f, data[0].keys())
        w.writeheader()
        for datum in data:
            w.writerow(datum)

In [138]:
def getAllData(beginning, end):
    for i in reversed(range(beginning, end)):
        content = contentForIssue(i)
        data = None
        if (i < 60):
            data = dataForIssueBefore60(content, i) 
        elif (i < 103):
            data = dataForIssueBefore103(content, i) 
        else:
            data = dataForIssue(content, i)
        writeData(data, 'issue_{}.csv'.format(i))

In [139]:
# 500 errors: 138, 108, 43
# Skipped b/c unusual: 30
getAllData(1, 4)

http://androidweekly.net/issues/issue-3
Found 9 items in 5 sections
http://androidweekly.net/issues/issue-2
Found 9 items in 5 sections
http://androidweekly.net/issues/issue-1
Found 9 items in 5 sections
